### 0. Инициализация

In [1]:
# Загружаем библиотеки необходимые для выполнения кода ноутбука.

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import logging

In [2]:
# Настройка форматов
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format',  '{:,.2f}'.format)

# Отключаем INFO сообщения от matplotlib
logging.getLogger('matplotlib').setLevel(logging.WARNING)

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['axes.titlesize'] = 'large'
plt.rcParams['axes.labelsize'] =  'large'
plt.rcParams['xtick.labelsize'] =  'medium'
plt.rcParams['ytick.labelsize'] =  'medium'
plt.rcParams['legend.title_fontsize'] = 'large'
plt.rcParams['legend.fontsize'] = 'medium'
plt.rcParams['legend.framealpha'] = .9

### 1. Первый этап

#### 1.1 Загрузка первичных данных

In [3]:
tracks = pd.read_parquet('data/tracks.parquet')
cat_names = pd.read_parquet('data/catalog_names.parquet')
interactions = pd.read_parquet('data/interactions.parquet')

#### 1.2 Обзор данных

##### 1.2.1 cat_names - справочник каталожных имен

In [4]:
# Типы данных
cat_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB


In [5]:
# Диапазон значений id
cat_names['id'].describe()[['min', 'max', 'count']].astype('int')

min              0
max      101521819
count      1812471
Name: id, dtype: int64

In [6]:
# Сожмем до int32 для экономии места
cat_names['id'] = cat_names['id'].astype('int32')

In [7]:
# Визуальный осмотр данных
cat_names.head(3)

,id,type,name
0,3,album,Taller Children
1,12,album,Wild Young Hearts
2,13,album,Lonesome Crow


In [8]:
# Уникальные типы каталожных единиц
cat_names['type'].value_counts()

type
track     1000000
album      658724
artist     153581
genre         166
Name: count, dtype: int64

In [9]:
# Проверка на пропуски -> отсутствуют
cat_names.isna().sum()

id      0
type    0
name    0
dtype: int64

In [10]:
# Наличие дубликатов по полю 'id' -> присутствуют
cat_names[['id']].duplicated().sum()

35774

In [11]:
# Наличие дубликатов по связке 'id', 'type' -> отсутсвуют
cat_names[['id', 'type']].duplicated().sum()

0

In [12]:
# Наличие дубликатов по связке 'type', 'name' -> присутствуют, и много
cat_names[['type', 'name']].duplicated().sum()

689769

In [13]:
# Наличие дубликатов по связке 'type', 'name' для разных типов
cat_type_name_duplicated = cat_names[cat_names[['type', 'name']].duplicated()]

for type, total_count in cat_names['type'].value_counts().to_dict().items():
    duplicated_count = len(cat_type_name_duplicated.query('type == @type'))
    display(
        f'Type [{type}] : '
        f'duplicated names {duplicated_count} of {total_count} '
        f'or {duplicated_count / total_count * 100:.1f}%.'
    )

del cat_type_name_duplicated, type, duplicated_count, total_count

'Type [track] : duplicated names 469276 of 1000000 or 46.9%.'

'Type [album] : duplicated names 218108 of 658724 or 33.1%.'

'Type [artist] : duplicated names 2385 of 153581 or 1.6%.'

'Type [genre] : duplicated names 0 of 166 or 0.0%.'

__NB! В каталоге около половины треков и трети альбомов имеют повторяющиеся (дублированные) имена.__   
Решение: оставляем как есть, корректировку (слияние) дубликатов в справочниках пока не проводим.

##### 1.2.2 tracks - данные о треках

In [14]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB


In [15]:
# Сжимаем 'track_id' до int32
tracks['track_id'] = tracks['track_id'].astype('int32')

In [16]:
# Исходное число треков
track_count_initial = len(tracks)

In [17]:
# визуальный отсмотр
tracks.head(3)

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]


In [18]:
# дубликаты по 'track_id' -> отсутствуют
tracks['track_id'].duplicated().sum()

0

In [19]:
# Все ли track_id есть в справочнике -> да, все
set(tracks['track_id']) - set(cat_names.query('type == "track"')['id'])

set()

In [20]:
# все ли id альбомов есть в справочнике -> нет, не все: 
# встречаются пропуски (nan)
(
    set(tracks['albums'].explode().drop_duplicates()) 
    - set(cat_names.query('type == "album"')['id'])
)

{nan}

In [21]:
# визуальный осмотр битых идентификаторов альбомов
track_ids_with_broken_albums = (
    tracks
    .explode('albums')
    .query('albums.isna()')
    ['track_id']
    .drop_duplicates()
)
print(
    f'Tracks with broken "albums": {len(track_ids_with_broken_albums)} '
    f'or {len(track_ids_with_broken_albums) / track_count_initial :.2%} '
    'of total.\n\nSample:'
)
tracks.query('track_id in @track_ids_with_broken_albums').head(3)

Tracks with broken "albums": 18 or 0.00% of total.

Sample:


,track_id,albums,artists,genres
310821,20200372,[],[],[]
310826,20200380,[],[],[]
312469,20305116,[],[],[]


__Проблема__: в данных о треках `tracks` есть 18 треков с отсутствующими (пустыми) значениями id альбомов.      
_Решение_: удаляем такие треки из справочника (каталога)

In [22]:
# удаляем треки с битыми (пустыми) альбомами
tracks = tracks.query('track_id not in @track_ids_with_broken_albums')

del track_ids_with_broken_albums

In [23]:
# все ли id артистов есть в справочнике -> нет, не все: 
# встречаются пропуски (nan)
(
    set(tracks['artists'].explode().drop_duplicates()) 
    - set(cat_names.query('type == "artist"')['id'])
)

{nan}

In [24]:
# визуальный осмотр битых идентификаторов артистов
track_ids_with_broken_artists = (
    tracks
    .explode('artists')
    .query('artists.isna()')
    ['track_id']
    .drop_duplicates()
)
print(
    f'Tracks with broken "artists": {len(track_ids_with_broken_artists)} '
    f'or {len(track_ids_with_broken_artists) / track_count_initial :.2%} '
    'of total.\n\nSample:'
)
tracks.query('track_id in @track_ids_with_broken_artists').head(3)

Tracks with broken "artists": 15351 or 1.54% of total.

Sample:


,track_id,albums,artists,genres
146031,3599314,[389925],[],"[68, 86]"
146163,3599591,[389944],[],"[68, 86]"
177544,4790215,[533785],[],"[68, 86]"


__Проблема__: в данных о треках `tracks` есть ~1.5% треков с отсутствующими (пустыми) значениями id артистов.      
_Решение_: удаляем такие треки из справочника (каталога)

In [25]:
# удаляем треки с битыми (пустыми) идентификаторами артистов
tracks = tracks.query('track_id not in @track_ids_with_broken_artists')

del track_ids_with_broken_artists

In [26]:
# все ли id жанров есть в справочнике -> нет, не все: 
# встречаются как пропуски (nan) так и безимянные жанры
genre_ids_missing = (
    set(tracks['genres'].explode().drop_duplicates()) 
    - set(cat_names.query('type == "genre"')['id'])
)
print(*genre_ids_missing)

130 131 132 133 134 135 nan 146 148 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 124 126


In [27]:
# визуальный осмотр битых идентификаторов жанров
track_ids_with_broken_genres = (
    tracks
    .explode('genres')
    .query('genres in @genre_ids_missing') 
    ['track_id']
    .drop_duplicates()
)
print(
    f'Tracks with broken "genres": {len(track_ids_with_broken_genres)} '
    f'or {len(track_ids_with_broken_genres) / track_count_initial :.2%} '
    'of total.\n\nSample:'
)
tracks.query('track_id in @track_ids_with_broken_genres').head(3)

Tracks with broken "genres": 51967 or 5.20% of total.

Sample:


,track_id,albums,artists,genres
36,436,[36],[330],"[28, 164]"
59,594,"[54, 88, 5479, 5785124, 9198099, 9231427, 1088...",[533],"[28, 162]"
125,1025,"[94, 2325, 8757, 8986, 318695, 17004129]",[937],"[28, 162]"


__Проблема__: в данных о треках `tracks` есть ~5% треков с неизвестными или отсутствующими (пустыми) значениями id жанров.      
_Решение_: удаляем такие треки из справочника (каталога)

In [28]:
# удаляем треки с неизвестными иои пустыми идентификаторами жанров
tracks = tracks.query('track_id not in @track_ids_with_broken_genres')

del genre_ids_missing, track_ids_with_broken_genres

In [29]:
# сколько треков осталось после чистки данных
print(
    f'Cleaned tracks: {len(tracks)} '
    f'or {len(tracks) / track_count_initial:.2%} of inital count.'
)

Cleaned tracks: 932664 or 93.27% of inital count.


In [30]:
# Приборка
del track_count_initial

##### 1.2.3 interactions - взаимодействия пользователей с треками

In [31]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   int16         
 3   started_at  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(2)
memory usage: 5.4 GB


In [32]:
# исходное число записей о взаимодействиях
interactions_count_initial = len(interactions)

In [33]:
# визуальный осмотр данных
interactions.head(3)

,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21


In [34]:
# Оценим кол-во взаимодействий с треками отсутствующими в справочнике tracks

# ids известных треков
known_track_ids = set(tracks['track_id'])

# взаимодействия с неизвестными треками
interactions_with_unknown_tracks = (
    interactions.query("track_id not in @known_track_ids")
)
print(
    'Interactions with unknown tracks: '
    f'{len(interactions_with_unknown_tracks) / interactions_count_initial:.2%} '
    'of total.'
)
del interactions_with_unknown_tracks

Interactions with unknown tracks: 3.55% of total.


Около 3% взаимодействий приходятся на треки, отсутствующие в каталоге (справочнике) `tracks`.       
_Решение_ Удаляем такие прослушивания из `interactions`

In [35]:
# Удаляем данные о прослушивании 'неизвестных' (или 'битых') треков
interactions = interactions.query('track_id in @known_track_ids')

del known_track_ids

In [36]:
# сколько взаимодействий осталось после чистки данных
print(
    f'Cleaned interactions: {len(interactions)} '
    f'or {len(interactions) / interactions_count_initial:.2%} of inital count.'
)

Cleaned interactions: 214720738 or 96.45% of inital count.


In [37]:
# Приборка
del interactions_count_initial

#### 1.3 Выводы

Проблемы с данными в исходных датасетах:
- в каталоге `cat_names` около половины(!) треков и трети альбомов имеют повторяющиеся (дублированные) имена,    
    - _Решение_: оставляем как есть, корректировку (слияние) дубликатов в справочниках пока не проводим.
- в данных о треках `tracks` есть 18 треков с отсутствующими (пустыми) значениями id альбомов,
    - _Решение_: удаляем такие треки из справочника (каталога)
- в данных о треках `tracks` есть ~1.5% треков с отсутствующими (пустыми) значениями id артистов.      
    - _Решение_: удаляем такие треки из справочника (каталога)
- в данных о треках `tracks` есть ~5% треков с неизвестными или отсутствующими (пустыми) значениями id жанров.      
    - _Решение_: удаляем такие треки из справочника (каталога)
- Около 3% взаимодействий приходятся на треки, отсутствующие в 'почищенном' каталоге (справочнике) `tracks`.       
    - _Решение:_ Удаляем такие прослушивания из `interactions`
- очевидных проблем с идентификаторами (ids) не обнаружено

### 2. Второй этап

#### 2.1 Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

In [38]:
# Создадим объеккт items
items = tracks.copy().rename(columns={'track_id': 'item_id'})

In [39]:
# Добавим к items строковое имя трека из каталога
items = items.join(
    cat_names.query('type == "track"').set_index('id')['name'],
    on='item_id'
).rename(columns={'name': 'item_name'})

# Визуальная проверка
items.head(3)

,item_id,albums,artists,genres,item_name
0,26,"[3, 2490753]",[16],"[11, 21]",Complimentary Me
1,38,"[3, 2490753]",[16],"[11, 21]",Momma's Boy
2,135,"[12, 214, 2490809]",[84],[11],Atticus


In [40]:
# Добавим к items имена альбомов, артистов и жанров из каталога

# Создаем словари типа {id: строковое_имя}
albums_names = (cat_names.query('type == "album"').set_index('id')['name']
               .to_dict())
artists_names = (cat_names.query('type == "artist"').set_index('id')['name']
                 .to_dict())
genres_names = (cat_names.query('type == "genre"').set_index('id')['name']
               .to_dict())

# Векторизуем функции поиска по словарю для использования с np.array
get_albums_names = np.vectorize(lambda x: albums_names[x])
get_artists_names = np.vectorize(lambda x: artists_names[x])
get_genres_names = np.vectorize(lambda x: genres_names[x])

items['albums_names'] = items['albums'].map(get_albums_names)
items['artists_names'] = items['artists'].map(get_artists_names)
items['genres_names'] = items['genres'].map(get_genres_names)

del (albums_names, artists_names, genres_names, get_albums_names,
     get_artists_names, get_genres_names)

In [41]:
# Визуальна проверка
items.head()

,item_id,albums,artists,genres,item_name,albums_names,artists_names,genres_names
0,26,"[3, 2490753]",[16],"[11, 21]",Complimentary Me,"[Taller Children, Taller Children]",[Elizabeth & the Catapult],"[pop, folk]"
1,38,"[3, 2490753]",[16],"[11, 21]",Momma's Boy,"[Taller Children, Taller Children]",[Elizabeth & the Catapult],"[pop, folk]"
2,135,"[12, 214, 2490809]",[84],[11],Atticus,"[Wild Young Hearts, Wild Young Hearts, Wild Yo...",[Noisettes],[pop]
3,136,"[12, 214, 2490809]",[84],[11],24 Hours,"[Wild Young Hearts, Wild Young Hearts, Wild Yo...",[Noisettes],[pop]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11],Don't Upset The Rhythm (Go Baby Go),"[Wild Young Hearts, Wild Young Hearts, Don't U...",[Noisettes],[pop]


In [45]:
interactions.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 214720738 entries, 0 to 291
Columns: 4 entries, user_id to started_at
dtypes: datetime64[ns](1), int16(1), int32(2)
memory usage: 5.2 GB


#### 2.1 EDA

In [ ]:
interactions['user_id'].nunique()

Общее кол-во уникальных пользователей в выборке около 1.3 млн.

In [ ]:
# Кол-во прослушанных треков по пользователям
track_count_by_user = (
    interactions.groupby('user_id').agg(tracks_listened=('track_id', 'count'))
)

In [ ]:
# Статистики по кол-ву прослушаных треков
track_count_by_user.describe().astype('int')

In [ ]:
ax = sns.histplot(track_count_by_user, stat='percent', binwidth=5)
ax.set_xlim(0, 300)
ax.axvline(track_count_by_user['tracks_listened'].median(), color='blue',
           linestyle='-', label='Median')
ax.axvline(track_count_by_user['tracks_listened'].mean(), color='red',
           linestyle='-', label='Mean')
ax.legend()
ax.set_title('Распределение количества прослушанных пользователем треков');

Медианное кол-ва прослушанных пользователем треков 54.      
Из-за наличия достаточного кол-ва меломанов с большим числом прослушанных треков (>1000) среднее существенно выше и составляет 156.

In [ ]:
# 100 нааиболее популярных треков
top100_popular = interactions.groupby('track_id').agg(users_listened=('user_id', 'count')).sort_values(by='users_listened', ascending=False)[:100].reset_index()
top100_popular = top100_popular.merge(
    cat_names.query('type == "track"')[['id', 'name']],
    left_on='track_id',
    right_on='id',
    how='left'
)

In [ ]:
top100_popular

Наиболее популярные жанры

Треки, которые никто не прослушал

#### Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

#### Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

### 3. Третий этап

#### 3.1 Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

#### 3.2 Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

#### 3.3 Топ популярных

Рассчитаем рекомендации как топ популярных.

#### 3.4 Персональные

Рассчитаем персональные рекомендации.

#### 3.5 Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

#### 3.6 Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

#### 3.7 Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

#### 3.8 Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

#### 3.9 Выводы, метрики

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.